In [1]:
import numpy as np 
import pandas as pd

In [2]:
df=pd.read_csv(r'C:\Users\rejee\Desktop\Agitation.csv')

In [3]:
df

,timestamp (ET),agitimestamp (ET),Location,level,Observation,Behaviour
0,12/10/2016 21:02,12/10/2016 21:00,Kitchen,5,Withdrawn,Normal
1,12/15/2016 3:22,12/15/2016 3:21,Kitchen,3,"Vocal1, Withdrawn",Normal
2,12/15/2016 19:14,12/16/2016 2:20,Other,3,Withdrawn,Normal
3,12/19/2016 1:57,12/19/2016 1:55,Other,4,Withdrawn,Normal
4,12/28/2016 20:36,12/28/2016 19:00,Other,4,"Vocal2, Withdrawn",Normal
...,...,...,...,...,...,...
307,7/29/2017 10:42,7/29/2017 10:42,Kitchen,1,Repetition;,Normal
308,08/03/2017 12:45,08/03/2017 12:44,Kitchen,1,Repetition;,Normal
309,08/04/2017 22:13,08/04/2017 22:13,Kitchen,1,Repetition;,Normal
310,08/07/2017 11:44,08/07/2017 11:44,Kitchen,1,Repetition;,Normal


In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(df.Observation, df.Behaviour, test_size = 0.2, stratify = df.Behaviour
                                                 ,random_state = 42)

In [5]:
df['word_length'] = df['Observation'].apply(lambda x:len(x.split()))

In [6]:
from keras.preprocessing.text import Tokenizer
num_words = 10000 # this means 15000 unique words can be taken 
tokenizer=Tokenizer(num_words,lower=True)
df_total = pd.concat([X_train, X_test], axis = 0)
tokenizer.fit_on_texts(df_total)

In [7]:
len(tokenizer.word_index)

14

In [8]:
df.word_length.max()

7

In [9]:
from keras_preprocessing.sequence import pad_sequences

X_train_ =tokenizer.texts_to_sequences(X_train)
X_train_pad=pad_sequences(X_train_,maxlen=171,padding='post')
X_test_ = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_, maxlen = 171, padding = 'post')

In [10]:
print(X_train_pad.shape,X_test_pad.shape) # this is our 2D matrix we can take this as Input data

(249, 171) (63, 171)


In [11]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow

In [12]:
from tensorflow.keras.layers import Dropout

In [13]:
import gensim.downloader as api
glove_gensim  = api.load('glove-wiki-gigaword-100') # this would download vector with 100 dimension

In [14]:
glove_gensim['cat'].shape[0] # this is the diemnsion of the vectors

100

In [21]:
from tensorflow.compat.v1.keras.layers import CuDNNLSTM,CuDNNGRU

In [15]:
vector_size = 100
gensim_weight_matrix = np.zeros((num_words ,vector_size))
gensim_weight_matrix.shape

for word, index in tokenizer.word_index.items():
    if index < num_words: # since index starts with zero 
        if word in glove_gensim.key_to_index:
            gensim_weight_matrix[index] = glove_gensim[word]
        else:
            gensim_weight_matrix[index] = np.zeros(100)

In [16]:
gensim_weight_matrix.shape

(10000, 100)

In [22]:
 #designing our architecture 
EMBEDDING_DIM = 100 # this means the embedding layer will create  a vector in 100 dimension
model_gensim = Sequential()
model_gensim.add(Embedding(input_dim = num_words,# the whole vocabulary size 
                          output_dim = EMBEDDING_DIM, # vector space dimension
                          input_length= X_train_pad.shape[1], # max_len of text sequence
                          weights = [gensim_weight_matrix],trainable = False))
model_gensim.add(Dropout(0.2))
model_gensim.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model_gensim.add(Dropout(0.2))
model_gensim.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model_gensim.add(Dropout(0.2))
model_gensim.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
model_gensim.add(Dense(1, activation = 'sigmoid'))
model_gensim.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics = 'accuracy')

In [23]:
model_gensim.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 171, 100)          1000000   
                                                                 
 dropout_3 (Dropout)         (None, 171, 100)          0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 171, 200)         161600    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 171, 200)          0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 171, 400)         643200    
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 171, 400)         

In [24]:
from keras.callbacks import EarlyStopping, ModelCheckpoint


es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('./model_gensim.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

In [25]:
history_gensim = model_gensim.fit(X_train_pad,y_train, epochs = 25, batch_size = 120, validation_data=(X_test_pad, y_test),verbose = 1, callbacks= [es, mc]  )

Epoch 1/25


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNNV2' used by {{node sequential_1/bidirectional_3/forward_cu_dnnlstm/CudnnRNNV2}} with these attrs: [seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", seed2=0, is_training=true]
Registered devices: [CPU]
Registered kernels:
  <no registered kernels>

	 [[sequential_1/bidirectional_3/forward_cu_dnnlstm/CudnnRNNV2]] [Op:__inference_train_function_21924]